In [1]:
import os
import nibabel as nib
from nilearn import glm
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy.io import loadmat 
import h5py
from scipy.signal import convolve2d
from scipy.signal import convolve
%matplotlib inline

/home/jmlee/.conda/envs/encoding/lib/python3.7/site-packages/nilearn/glm/__init__.py:56: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  'It may change in any future release of Nilearn.', FutureWarning)


In [2]:
layername = ['/conv1','/conv2','/conv3','/conv4','/conv5','/fc6','/fc7','/fc8']
dataroot = '/local_raid3/03_user/jungmin/02_data/Encoding_decoding/encoding_analyzing/02_Goal-Driven-DL/01_NeuralEncodingDecoding/sourcecode/source_code_v2/source_code_v2/feature_extracted/'
saveroot = '/local_raid3/03_user/jungmin/02_data/Encoding_decoding/encoding_analyzing/02_Goal-Driven-DL/01_NeuralEncodingDecoding/sourcecode/source_code_v2/source_code_v2/jmlee/'

# calculate the temporal mean 

In [9]:
for lay in range(1,len(layername)):
    N = 0
    for seg in range (1,18):
        secpath = dataroot + 'AlexNet_feature_maps_seg'+ str(seg)+'.h5'
        secpath = dataroot + 'AlexNet_feature_maps_seg'+ str(seg)+'.h5'
        if os.path.isfile(secpath):
            lay_feat = h5py.File(secpath, 'r')
            lay_feat = lay_feat[layername[lay] + '/data']            
            lay_feat = lay_feat.value
            dim = lay_feat.shape
            dim = np.asarray(dim)
            print(dim.shape)
            if seg == 1: 
                lay_feat_mean = np.zeros((dim[1:],1))
            lay_feat_mean = lay_feat_mean + sum(lay_feat,len(dim))
            N = N  + dim
    lay_feat_mean = lay_feat_mean/N
    foldpath = saveroot + 'AlexNet_feature_maps_avg_2' + str(lay)+'.h5'
    f = h5py.File(foldpath, 'w')
    g = f.create_group(layername[lay])
    data = g.create_dataset("data", data=lay_feat_mean)
    print("Saved: {}".format(foldpath))
    f.close()
    
    # h5create([dataroot,'AlexNet_feature_maps_avg.h5'],[layername{lay},'/data'],...
    #     [size(lay_feat_mean)],'Datatype','single');
    # h5write([dataroot,'AlexNet_feature_maps_avg.h5'], [layername{lay},'/data'], lay_feat_mean);

(4,)


/home/jmlee/.conda/envs/encoding/lib/python3.7/site-packages/ipykernel_launcher.py:9: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  if __name__ == "__main__":


TypeError: only integer scalar arrays can be converted to a scalar index

In [10]:
dir = '/local_raid3/03_user/jungmin/02_data/Encoding_decoding/encoding_analyzing/02_Goal-Driven-DL/01_NeuralEncodingDecoding/sourcecode/source_code_v2/source_code_v2/feature_extracted/AlexNet_feature_maps_avg.h5'
feature_avg = h5py.File(dir , 'r')
print(feature_avg['/conv1/data'].shape)

(96, 55, 55)


In [ ]:
a_list = []
for i in range(100):
    r = np.corrcoef(lay_feat_mean[:,:,i].ravel(), feature_avg[:,:,i].ravel())[0,1]
    a_list.append(r)
    
print(np.mean(a_list))
# print(np.array_equal(lay_feat_mean, feature_avg))

# calculate the temporal standard deviation

In [ ]:
for lay in range(1,len(layername)):
    lay_feat_mean = h5py.File(secpath + 'AlexNet_feature_maps_avg.h5' + layername(lay) + '/data', 'r')
    N = 0
    for seg in range (1,18):
        print('Seg: ',  str(seg) , '; Layer: ',layername(lay))
        secpath = dataroot + 'AlexNet_feature_maps_seg'+ str(seg)+'.h5'
        if os.path.isfile(secpath):   
            lay_feat = h5py.File(secpath + layername(lay) + '/data', 'r')
            lay_feat = np.apply_along_axis(lay_feat_mean - lay_feat)
            lay_feat = lay_feat **2 
            dim = lay_feat.shape
            if seg == 1: 
                lay_feat_mean = np.zeros([dim[1:1:-1]])
            lay_feat_std = lay_feat_std + sum(lay_feat,len(dim))
            N = N  + dim
    lay_feat_std = np.sqrt(lay_feat_std/(N-1)) 
    lay_feat_std[lay_feat_std==0] = 1
    foldpath = dataroot + 'AlexNet_feature_maps_std' + str(lay)+'.h5'
    f = h5py.File(foldpath, 'w')
    g = f.create_group(layername[lay])
    data = g.create_dataset("data", data= lay_feat_std)
    print("Saved: {}".format(foldpath))
    f.close()
    
    # lay_feat_std(lay_feat_std==0) = 1;
    # h5create([dataroot,'AlexNet_feature_maps_std.h5'],[layername{lay},'/data'],...
    #     [size(lay_feat_mean)],'Datatype','single');
    # h5write([dataroot,'AlexNet_feature_maps_std.h5'], [layername{lay},'/data'], lay_feat_std);


# SVD-updating algorithm

In [ ]:
Niter = 2 # number of iteration to compute principle component

In [5]:
from jlee_amri_sig_isvd import *

In [ ]:
for lay in range(1,len(layername)):
    lay_feat_mean = h5py.File(secpath + 'AlexNet_feature_maps_avg.h5' + layername(lay) + '/data', 'r')
    lay_feat_std = h5py.File(secpath + 'AlexNet_feature_maps_std.h5' + layername(lay) + '/data', 'r')
    
    k0 = 0
    percp = 0.99
    for iter in range(1,Niter): 
        for seg in range (1,18):
            print(['Layer: ', str(lay),'; Seg: ',str(seg),'; Comp:',str(k0)])
            secpath = dataroot + 'AlexNet_feature_maps_seg'+ str(seg)+'.h5'
            if os.path.isfile(secpath)==2:
                lay_feat = h5py.File(secpath + layername(lay) + '/data', 'r')
                lay_feat = np.subtract(lay_feat, lay_feat_mean)
                lay_feat = np.divide(lay_feat, lay_feat_std)
                np.nan_to_num(lay_feat, copy=False)
                dim = lay_feat.shape
                lay_feat = lay_feat.reshape(np.prod(dim[:-1]),dim[-1],order='F').copy()
                # lay_feat = bsxfun(@minus, lay_feat, lay_feat_mean);
                # lay_feat = bsxfun(@rdivide, lay_feat, lay_feat_std);
                # lay_feat(isnan(lay_feat)) = 0; % assign 0 to nan values
                # dim = size(lay_feat);
                # lay_feat = reshape(lay_feat, prod(dim(1:end-1)),dim(end));
                if (seg == 1) and (iter == 1): 
                    [B, S, k0] = amri_sig_isvd(lay_feat, 'var', percp)
                else :
                    [B, S, k0] = amri_sig_isvd(lay_feat, 'var',  percp, 'init', {B,S})
                
            else :
                print('file doesn''t exist')
    s = np.diag(S)
    
    # save principal components
    foldpath = dataroot +'AlexNet_feature_maps_svd_layer' + str(lay)+'.h5'
    f = h5py.File(foldpath, 'w')
    g = f.create_group(layername[lay])
    data = g.create_dataset("B", data=B)
    data = g.create_dataset("S", data=S)
    print("Saved: {}".format(foldpath))
    f.close()

In [ ]:
layername = {'/conv1';'/conv2';'/conv3';'/conv4';'/conv5';'/fc6';'/fc7';'/fc8'}
dataroot = '/local_raid3/03_user/jungmin/02_data/Encoding_decoding/encoding_analyzing/02_Goal-Driven-DL/01_NeuralEncodingDecoding/sourcecode/source_code_v2/source_code_v2/feature_extracted/'

In [ ]:
srate = 30
p  = [5, 16, 1, 1, 6, 0, 32]
tr = 1/srate


In [ ]:
import scipy.stats
import numpy as N

def spm_hrf(TR,p=[5,16,1,1,6,0,32]):
    """ An implementation of spm_hrf.m from the SPM distribution
Arguments:
Required:
TR: repetition time at which to generate the HRF (in seconds)
Optional:
p: list with parameters of the two gamma functions:
                                                     defaults
                                                    (seconds)
   p[0] - delay of response (relative to onset)         6
   p[1] - delay of undershoot (relative to onset)      16
   p[2] - dispersion of response                        1
   p[3] - dispersion of undershoot                      1
   p[4] - ratio of response to undershoot               6
   p[5] - onset (seconds)                               0
   p[6] - length of kernel (seconds)                   32
"""

    p=[float(x) for x in p]

    fMRI_T = 16.0

    TR=float(TR)
    dt  = TR/fMRI_T
    u   = N.arange(p[6]/dt + 1) - p[5]/dt
    hrf=scipy.stats.gamma.pdf(u,p[0]/p[2],scale=1.0/(dt/p[2])) - scipy.stats.gamma.pdf(u,p[1]/p[3],scale=1.0/(dt/p[3]))/p[4]
    good_pts=N.array(range(N.int(p[6]/TR)))*int(fMRI_T)
    hrf=hrf[list(good_pts)]    
    hrf = hrf/N.sum(hrf)
    return hrf

In [ ]:
poldrack_hrf = spm_hrf(tr,p)
print(poldrack_hrf.shape)

poldrack_hrf = np.expand_dims(poldrack_hrf, -1)
print(poldrack_hrf.shape)

print(poldrack_hrf.dtype)
poldrack_hrf = np.array(poldrack_hrf, dtype=np.float32)
print(poldrack_hrf.dtype)

In [ ]:
for seg in range(1,18):
    secpath = dataroot + 'AlexNet_feature_maps_seg'+ str(seg)+'.h5'
    if (secpath,'file')==2 :
            lay_feat = h5py.File(secpath + layername(lay) + '/data', 'r')
            lay_feat = np.subtract(lay_feat, lay_feat_mean)
            lay_feat = np.divide(lay_feat, lay_feat_std)
            np.nan_to_num(lay_feat, copy=False)
            dim = lay_feat.shape
            lay_feat = lay_feat.reshape(np.prod(dim[:-1]),dim[-1],order='F').copy() 
            Y = lay_feat*B/np.sqrt((B,1).shape) #Y: #time-by-#components
            Nf = (Y,1).shape
            ts = convolve(poldrack_hrf, lay_feat)
            ts = ts[4*srate+1:4*srate+Nf, :]
            down_sample_idx = np.arange(srate+1, len(ts), srate*2)
            ts = ts[down_sample_idx]
            print("Down sampled ts: {}".format(ts.shape))
            
            foldpath = dataroot+'AlexNet_feature_maps_pcareduced_seg' + str(seg)+'.h5'
            f = h5py.File(foldpath, 'w')
            g = f.create_group(layername[lay])
            data = g.create_dataset("data", data=ts)
            print("Saved: {}".format(foldpath))
            f.close()

In [ ]:
for seg in range(1,18):
    secpath = dataroot + 'AlexNet_feature_maps_test'+ str(seg)+'.h5'
    if (secpath,'file')==2 :
            lay_feat = h5py.File(secpath + layername(lay) + '/data', 'r')
            lay_feat = np.subtract(lay_feat, lay_feat_mean)
            lay_feat = np.divide(lay_feat, lay_feat_std)
            np.nan_to_num(lay_feat, copy=False)
            dim = lay_feat.shape
            lay_feat = lay_feat.reshape(np.prod(dim[:-1]),dim[-1],order='F').copy() 
            Y = lay_feat*B/np.sqrt((B,1).shape) #Y: #time-by-#components
            Nf = (Y,1).shape
            ts = convolve(poldrack_hrf, lay_feat)
            ts = ts[4*srate+1:4*srate+Nf, :]
            down_sample_idx = np.arange(srate+1, len(ts), srate*2)
            ts = ts[down_sample_idx]
            print("Down sampled ts: {}".format(ts.shape))
            
            foldpath = dataroot+'AlexNet_feature_maps_pcareduced_test' + str(seg)+'.h5'
            f = h5py.File(foldpath, 'w')
            g = f.create_group(layername[lay])
            data = g.create_dataset("data", data=ts)
            print("Saved: {}".format(foldpath))
            f.close()

In [ ]:
layername = {'/conv1';'/conv2';'/conv3';'/conv4';'/conv5';'/fc6';'/fc7';'/fc8'}
dataroot = '/local_raid3/03_user/jungmin/02_data/Encoding_decoding/encoding_analyzing/02_Goal-Driven-DL/01_NeuralEncodingDecoding/sourcecode/source_code_v2/source_code_v2/feature_extracted/'

In [ ]:
for lay in len(layername):
    for seg in range(1,18):
            secpath = dataroot + 'AlexNet_feature_maps_pcareduced_seg'+ str(seg)+'.h5'     
            lay_feat = h5py.File(secpath + layername(lay) + '/data', 'r')
            dim = lay_feat.shape
            Nf = dim[0] # number of frames
            if seg == 1:
                Y = np.zeros([Nf*18, dim(2)],'single') 
            Y[(seg-1)*Nf+1:seg*Nf, :] = lay_feat
    
    foldpath = dataroot+'AlexNet_feature_maps_pcareduced_concatenated.h5' + str(seg)+'.h5'
    f = h5py.File(foldpath, 'w')
    g = f.create_group(layername[lay])
    data = g.create_dataset("data", data=Y)
    print("Saved: {}".format(foldpath))
    f.close()
